In [1]:
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.models import build_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np

classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
# config file 들고오기
cfg = Config.fromfile('/opt/ml/realcode/cascade_mask_rcnn_x101_fpn_1x_trash.py')

PREFIX = '/opt/ml/input/data/'

epoch = 12

# dataset 바꾸기
cfg.data.train.classes = classes
cfg.data.train.img_prefix = PREFIX
cfg.data.train.ann_file = PREFIX + 'train.json'
cfg.data.train.pipeline[2]['img_scale'] = (512, 512)

cfg.data.val.classes = classes
cfg.data.val.img_prefix = PREFIX
cfg.data.val.ann_file = PREFIX + 'val.json'
cfg.data.val.pipeline[1]['img_scale'] = (512, 512)

cfg.data.test.classes = classes
cfg.data.test.img_prefix = PREFIX
cfg.data.test.ann_file = PREFIX + 'test.json'
cfg.data.test.pipeline[1]['img_scale'] = (512, 512)

cfg.data.samples_per_gpu = 4

cfg.seed=2020
cfg.gpu_ids = [0]
cfg.work_dir = '/opt/ml/realcode/work_dirs/cascade_rcnn_x101_fpn_1x_trash'

# cfg.model.roi_head.bbox_head.num_classes = 11

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None

# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, f'epoch_{epoch}.pth')

dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

output = single_gpu_test(model, data_loader, show_score_thr=0.05)

prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
imag_ids = coco.getImgIds()


/opt/conda/lib/python3.7/site-packages/mmcv/utils/registry.py:250: UserWarning: The old API of register_module(module, force=False) is deprecated and will be removed, please use the new API register_module(name=None, force=False, module=None) instead.
  'The old API of register_module(module, force=False) '


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


2021-05-14 14:56:55,943 - mmdet - INFO - load model from: open-mmlab://resnext101_32x4d
2021-05-14 14:56:55,947 - mmdet - INFO - Use load_from_openmmlab loader


Use load_from_local loader
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 837/837, 2.4 task/s, elapsed: 345s, ETA:     0sloading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [3]:
print(output[0])

([array([], shape=(0, 5), dtype=float32), array([[1.6645169e+02, 7.4975540e+01, 2.5428650e+02, 1.6588290e+02,
        5.1220137e-01],
       [2.3323289e+02, 1.5759534e+02, 2.5182079e+02, 2.3958292e+02,
        1.3505103e-01],
       [0.0000000e+00, 3.3629037e+02, 7.4538223e+01, 4.5377475e+02,
        9.4806015e-02]], dtype=float32), array([[1.6658017e+02, 7.5110573e+01, 2.5426059e+02, 1.6588449e+02,
        4.6446812e-01],
       [1.2174674e+02, 1.2843819e+01, 3.6410468e+02, 2.3813507e+02,
        5.4376066e-02]], dtype=float32), array([], shape=(0, 5), dtype=float32), array([], shape=(0, 5), dtype=float32), array([], shape=(0, 5), dtype=float32), array([[0.0000000e+00, 3.3632404e+02, 7.4400795e+01, 4.5374854e+02,
        9.6029855e-02]], dtype=float32), array([], shape=(0, 5), dtype=float32), array([[172.79912   , 238.60167   , 463.40448   , 467.66803   ,
          0.99824154],
       [  0.        , 336.4362    ,  75.0018    , 453.81778   ,
          0.47004235]], dtype=float32), arra

In [7]:

class_num = 11
for i, out in enumerate(output):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for j in range(class_num):
        for o in out[0][j]:
            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])


submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.work_dir, f'submission_{epoch}.csv'), index=None)
submission.head()

,PredictionString,image_id
0,1 0.51220137 166.45169 74.97554 254.2865 165.8...,batch_01_vt/0021.jpg
1,1 0.4486741 283.94955 345.35648 309.4296 376.8...,batch_01_vt/0028.jpg
2,1 0.7715703 74.71916 168.09293 373.2743 385.29...,batch_01_vt/0031.jpg
3,1 0.15881415 297.9738 159.75249 354.06448 226....,batch_01_vt/0032.jpg
4,1 0.38044408 291.4297 1.4255848 313.6103 23.70...,batch_01_vt/0070.jpg
